<a href="https://colab.research.google.com/github/DarkShadowKonoha/Facial_Emotion_Recognition/blob/main/Facial_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 23.21 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [3]:
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]

In [4]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [5]:
import os

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [6]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/', emotion))
  os.mkdir(os.path.join('master_data/testing/', emotion))

In [7]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/training', emotion, image)
    copyfile(source, dest)
  for image in test_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/testing', emotion, image)
    copyfile(source, dest)

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [25]:
model = tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape =(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(8, activation='softmax')
])
model.compile(optimizer = Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 49, 49, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 10, 10, 64)      

In [14]:
train_dir ='/content/master_data/training'
test_dir = '/content/master_data/testing'

train_datagen  = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100,100),
                                                    class_mode='categorical',
                                                    batch_size=120
                                                    )

test_datagen  = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100,100),
                                                    class_mode='categorical',
                                                    batch_size=120
                                                    )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [15]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)b

In [26]:
model.fit(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

Epoch 1/10
92/92 [==============================] - 18s 193ms/step - loss: 1.8494 - acc: 0.4731 - val_loss: 1.0609 - val_acc: 0.4158
Epoch 2/10
92/92 [==============================] - 16s 178ms/step - loss: 1.0492 - acc: 0.4961 - val_loss: 1.0425 - val_acc: 0.5011
Epoch 3/10
92/92 [==============================] - 16s 178ms/step - loss: 1.0454 - acc: 0.4934 - val_loss: 1.0471 - val_acc: 0.5011
Epoch 4/10
92/92 [==============================] - 16s 177ms/step - loss: 1.0413 - acc: 0.4907 - val_loss: 1.0418 - val_acc: 0.5011
